### How to Tune Batch Size and Number of Epochs

Use scikit-learn to grid search the batch size and epochs

The batch size in iterative gradient descent is the number of patterns shown to the network before the weights are updated.
It is also an optimization in the training of the network, defining how many patterns to read at a time and keep in memory.

The number of epochs is the number of times that the entire training dataset is shown to the network during training.
Some networks are sensitive to the batch size, such as LSTM recurrent neural networks and Convolutional Neural Networks.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using TensorFlow backend.


### Load dataset

In [2]:
df = pd.read_csv("../datasets/pima-indians-diabetes.csv", delimiter=",")

### Investigate data

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### Split data into input (X) and output (Y) variables

In [5]:
X = df.drop('Outcome', axis=1).as_matrix()
Y = df['Outcome'].as_matrix()

### Function to define model (required for KerasClassifier)

In [6]:
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

### Create model

In [7]:
model = KerasClassifier(build_fn=create_model, verbose=0)

### Define the grid search parameters

In [8]:
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)

### Run GridSearch

In [9]:
grid_result = grid.fit(X, Y)

### Summarize results

In [10]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.690104 using {'batch_size': 80, 'epochs': 100}
0.433594 (0.143773) with: {'batch_size': 10, 'epochs': 10}
0.559896 (0.114657) with: {'batch_size': 10, 'epochs': 50}
0.680990 (0.019488) with: {'batch_size': 10, 'epochs': 100}
0.466146 (0.137566) with: {'batch_size': 20, 'epochs': 10}
0.661458 (0.012890) with: {'batch_size': 20, 'epochs': 50}
0.666667 (0.034104) with: {'batch_size': 20, 'epochs': 100}
0.602865 (0.063335) with: {'batch_size': 40, 'epochs': 10}
0.667969 (0.044993) with: {'batch_size': 40, 'epochs': 50}
0.635417 (0.020752) with: {'batch_size': 40, 'epochs': 100}
0.613281 (0.045999) with: {'batch_size': 60, 'epochs': 10}
0.643229 (0.031466) with: {'batch_size': 60, 'epochs': 50}
0.664062 (0.051329) with: {'batch_size': 60, 'epochs': 100}
0.558594 (0.017758) with: {'batch_size': 80, 'epochs': 10}
0.671875 (0.043146) with: {'batch_size': 80, 'epochs': 50}
0.690104 (0.023073) with: {'batch_size': 80, 'epochs': 100}
0.546875 (0.048159) with: {'batch_size': 100, 'epochs':